Start by downloading all of the data files we need.

For the purpose of the MVP, the highest reported 7-day average of cases according to the nychealth data is that of the 7 days prior to 01/08/2021, so here we'll just download that approximate week from both 2021 and 2020, and the data from the nychealth GitHub.

In [11]:
import urllib.request

url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
#week_nums = [210320, 210313, 210306, 210227, 210220, 210213, 210206, 210130, 210123, 210116, 210109, 210102, 
#             201226, 201219, 201212, ]
week_nums = [210109, 200111]

for week_num in week_nums:
    urllib.request.urlretrieve(f"http://web.mta.info/developers/data/nyct/turnstile/turnstile_{week_num}.txt",
                               f"data/turnstile_{week_num}.txt")
urllib.request.urlretrieve(f"https://github.com/nychealth/coronavirus-data/blob/master/trends/data-by-day.csv",
                           f"data/data-by-day.csv")

('data/data-by-day.csv', <http.client.HTTPMessage at 0x7fc55560a9a0>)

In [12]:
from sqlalchemy import create_engine
import pandas as pd

#engine = create_engine("sqlite:///mta.db")
#all_data = pd.read_sql('SELECT * FROM mta_data;', engine)
#pd.read_sql('SELECT * FROM data-by-day LIMIT 10;', engine)

In [13]:
mta_data_1 = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_210320.txt')
covid_data = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv')

In [22]:
mta_data_1.columns = mta_data_1.columns.str.strip()
mta_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207054 entries, 0 to 207053
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   C/A       207054 non-null  object
 1   UNIT      207054 non-null  object
 2   SCP       207054 non-null  object
 3   STATION   207054 non-null  object
 4   LINENAME  207054 non-null  object
 5   DIVISION  207054 non-null  object
 6   DATE      207054 non-null  object
 7   TIME      207054 non-null  object
 8   DESC      207054 non-null  object
 9   ENTRIES   207054 non-null  int64 
 10  EXITS     207054 non-null  int64 
dtypes: int64(2), object(9)
memory usage: 17.4+ MB


In [23]:
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   date_of_interest          388 non-null    object
 1   CASE_COUNT                388 non-null    int64 
 2   PROBABLE_CASE_COUNT       388 non-null    int64 
 3   HOSPITALIZED_COUNT        388 non-null    int64 
 4   DEATH_COUNT               388 non-null    int64 
 5   PROBABLE_DEATH_COUNT      388 non-null    int64 
 6   CASE_COUNT_7DAY_AVG       388 non-null    int64 
 7   ALL_CASE_COUNT_7DAY_AVG   388 non-null    int64 
 8   HOSP_COUNT_7DAY_AVG       388 non-null    int64 
 9   DEATH_COUNT_7DAY_AVG      388 non-null    int64 
 10  ALL_DEATH_COUNT_7DAY_AVG  388 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 33.5+ KB


That's a lot of data, so for now we'll drop all of the columns that specify borough data.

In [18]:
covid_data.drop(covid_data.columns[range(11, 62)], axis = 1, inplace = True)

In [19]:
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   date_of_interest          388 non-null    object
 1   CASE_COUNT                388 non-null    int64 
 2   PROBABLE_CASE_COUNT       388 non-null    int64 
 3   HOSPITALIZED_COUNT        388 non-null    int64 
 4   DEATH_COUNT               388 non-null    int64 
 5   PROBABLE_DEATH_COUNT      388 non-null    int64 
 6   CASE_COUNT_7DAY_AVG       388 non-null    int64 
 7   ALL_CASE_COUNT_7DAY_AVG   388 non-null    int64 
 8   HOSP_COUNT_7DAY_AVG       388 non-null    int64 
 9   DEATH_COUNT_7DAY_AVG      388 non-null    int64 
 10  ALL_DEATH_COUNT_7DAY_AVG  388 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 33.5+ KB


In [24]:
covid_data.head()

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG
0,02/29/2020,1,0,1,0,0,0,0,0,0,0
1,03/01/2020,0,0,1,0,0,0,0,0,0,0
2,03/02/2020,0,0,2,0,0,0,0,0,0,0
3,03/03/2020,1,0,7,0,0,0,0,0,0,0
4,03/04/2020,5,0,2,0,0,0,0,0,0,0


In [23]:
mta_data_1.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,03:00:00,REGULAR,7543925,2573549
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,07:00:00,REGULAR,7543931,2573559
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,11:00:00,REGULAR,7543970,2573622
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,15:00:00,REGULAR,7544072,2573700
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,19:00:00,REGULAR,7544219,2573729


In [12]:
covid_data.CASE_COUNT.describe()

count     387.000000
mean     1759.418605
std      1682.616697
min         0.000000
25%       355.500000
50%      1026.000000
75%      3058.000000
max      6561.000000
Name: CASE_COUNT, dtype: float64

In [25]:
mask = (covid_data.HOSPITALIZED_COUNT > 1000)
covid_data[mask].head(10)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG
23,03/23/2020,3570,14,1147,82,13,3132,3138,703,39,44
24,03/24/2020,4500,12,1314,95,12,3425,3432,835,52,58
25,03/25/2020,4874,18,1411,124,18,3697,3705,966,66,75
26,03/26/2020,5046,41,1543,186,41,3888,3902,1096,90,103
27,03/27/2020,5117,41,1569,218,41,4046,4066,1213,114,133
28,03/28/2020,3480,51,1390,273,51,4167,4193,1306,147,172
29,03/29/2020,3562,63,1503,290,63,4307,4341,1411,181,215
30,03/30/2020,6130,85,1854,325,85,4673,4717,1512,216,260
31,03/31/2020,5458,85,1801,385,84,4810,4864,1582,257,312
32,04/01/2020,5448,101,1744,447,101,4892,4958,1629,303,370


In [27]:
covid_data.sort_values('ALL_DEATH_COUNT_7DAY_AVG', ascending = False).head(10)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG
42,04/11/2020,3732,171,1038,542,171,5010,5203,1424,566,759
43,04/12/2020,2887,175,957,571,175,4882,5076,1360,565,758
41,04/10/2020,4510,179,1289,539,179,5029,5225,1484,561,757
44,04/13/2020,3312,184,1156,570,184,4448,4636,1277,564,752
40,04/09/2020,5071,193,1350,559,193,5194,5388,1545,555,749
39,04/08/2020,5579,199,1566,568,199,5291,5481,1591,546,737
45,04/14/2020,4142,140,986,507,140,4176,4353,1192,551,728
38,04/07/2020,6042,215,1581,598,215,5272,5449,1617,529,705
46,04/15/2020,3876,105,869,453,105,3933,4097,1092,534,698
47,04/16/2020,3531,136,782,413,136,3713,3869,1011,514,669


How will we calculate the total daily traffic of a station? Get the MAX and MIN of both ENTRIES and EXITS for each day (grouped by station), calculate the difference of each, and add them together. This will give us a single number that can be used as a general measure of the flow of traffic for each station each day. We can then add the numbers for every station together to get the total amount of traffic for the entire MTA each day.

In [37]:
grouped_max_entries = mta_data_1.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"]).ENTRIES.max()
grouped_max_entries

C/A    UNIT  SCP       STATION        DATE      
A002   R051  02-00-00  59 ST          03/13/2021    7544310
                                      03/14/2021    7544559
                                      03/15/2021    7545056
                                      03/16/2021    7545597
                                      03/17/2021    7546147
                                                     ...   
TRAM2  R469  00-05-01  RIT-ROOSEVELT  03/15/2021       5554
                                      03/16/2021       5554
                                      03/17/2021       5554
                                      03/18/2021       5554
                                      03/19/2021       5554
Name: ENTRIES, Length: 34973, dtype: int64

In [38]:
grouped_min_entries = mta_data_1.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"]).ENTRIES.min()
grouped_min_entries

C/A    UNIT  SCP       STATION        DATE      
A002   R051  02-00-00  59 ST          03/13/2021    7543925
                                      03/14/2021    7544321
                                      03/15/2021    7544591
                                      03/16/2021    7545133
                                      03/17/2021    7545678
                                                     ...   
TRAM2  R469  00-05-01  RIT-ROOSEVELT  03/15/2021       5554
                                      03/16/2021       5554
                                      03/17/2021       5554
                                      03/18/2021       5554
                                      03/19/2021       5554
Name: ENTRIES, Length: 34973, dtype: int64

In [51]:
#mta_data_1["NEW_ENTRIES"] = grouped_max_entries - grouped_min_entries
new_entries = grouped_max_entries - grouped_min_entries
new_entries

C/A    UNIT  SCP       STATION        DATE      
A002   R051  02-00-00  59 ST          03/13/2021    385
                                      03/14/2021    238
                                      03/15/2021    465
                                      03/16/2021    464
                                      03/17/2021    469
                                                   ... 
TRAM2  R469  00-05-01  RIT-ROOSEVELT  03/15/2021      0
                                      03/16/2021      0
                                      03/17/2021      0
                                      03/18/2021      0
                                      03/19/2021      0
Name: ENTRIES, Length: 34973, dtype: int64

In [49]:
#test = mta_data_1
#test.append(new_entries)

In [ ]:
mta_data_1["NEW_EXITS"]

In [ ]:
mta_data_1["TOTAL_TRAFFIC"] #Add new entrances and exits to get total through traffic.